# Validation of PML method

In [ ]:
%env OMP_NUM_THREADS=1
import firedrake as fd
import numpy as np
import matplotlib.pyplot as plt
from scattering import solve, compute_error, plot_mesh, plot_field, plot_far_field
from mesh import generate_mesh

In [ ]:
# sound speed
c = 340
# point source inside obstacle
x0 = fd.Constant([0.5, 0])
# number of cells across PML
N = 2

In [ ]:
# fundamental solution
def exact_solution(mesh, k, x0):
    x = fd.SpatialCoordinate(mesh)
    z = k * fd.sqrt(fd.inner(x - x0, x - x0))
    u_re = -fd.bessel_Y(0, z) / 4
    u_im = fd.bessel_J(0, z) / 4
    return fd.as_vector([u_re, u_im])

# normal derivative of fundamental solution
def neumann_bc(mesh, k, x0):
    x = fd.SpatialCoordinate(mesh)
    n = fd.FacetNormal(mesh)
    z = k * fd.sqrt(fd.inner(x - x0, x - x0))
    dzdn = k / fd.sqrt(fd.inner(x - x0, x - x0)) * fd.dot(x - x0, n)
    dudn_re = fd.bessel_Y(1, z) / 4 * dzdn
    dudn_im = -fd.bessel_J(1, z) / 4 * dzdn
    return fd.as_vector([dudn_re, dudn_im])

In [ ]:
# shape of obstacle
# options: "circle", "kite", "square"
shape = "circle"
# set up PML
a0 = b0 = 2.0
a1 = b1 = 2.25
h0 = (a1 - a0) / N
cached_mesh = False

# generate mesh
if cached_mesh:
    mesh = fd.Mesh(shape + str(0) + ".msh")
else:
    mesh = generate_mesh(a0, a1, b0, b1, shape, N)

In [ ]:
# visualize mesh
plot_mesh(mesh)

In [ ]:
# angular frequency of source
omega = 250
# wavenumber of source
k = omega / c
# compute exact solution as Dirichlet boundary condition
u = exact_solution(mesh, k, x0)
# compute approximate solution
uh = solve(mesh, k, a0, a1, b0, b1, u)

In [ ]:
# visualize exact solution
print("Exact solution:")
W = fd.VectorFunctionSpace(mesh, "CG", 1)
plot_field(fd.interpolate(u, W), a0, a1, b0, b1)

In [ ]:
# visualize numerical solution
print("Numerical solution:")
plot_field(uh, a0, a1, b0, b1)

In [ ]:
# evaluate relative error in L2-norm
rel_err = compute_error(u, uh, quad_deg=4)
print(f"Relative error {rel_err:.2%}")

In [ ]:
# plot far field pattern evaluated by boundary-based formula 
plot_far_field(k, uh)

In [ ]:
# set up convergence test for scattered field
max_level = 4
levels = np.arange(max_level)
hs = h0 / 2**levels
omegas = [250, 750, 1250]

In [ ]:
# create mesh hierarchy by uniform refinement
mesh_hierarchy = []
if cached_mesh:
    for level in levels:
        mesh_hierarchy.append(fd.Mesh(shape + str(level) + ".msh"))
else:
    for level in levels:
        mesh_hierarchy.append(generate_mesh(a0, a1, b0, b1, shape, N, level))

In [ ]:
# test of Dirichlet problem and L2-norm error
neumann = False
for omega in omegas:
    print(f"angular freqency = {omega}")
    k = omega / c
    errors = []
    for level in levels:
        m = mesh_hierarchy[level]
        u = exact_solution(m, k, x0)
        uh = solve(m, k, a0, a1, b0, b1, u, neumann)
        rel_err = compute_error(u, uh, quad_deg=4)
        print(f"refinement level {level}, relative error {rel_err:.2%}")
        errors.append(rel_err)
    k = np.polyfit(np.log(hs), np.log(errors), 1)[0]
    print(f"convergence rate = {k:.2}")
    plt.loglog(hs, errors, "-o",
               label=r"Relative error of $\omega=$"+f"{omega}")
    print("----------------------------------------")

plt.loglog(hs, hs**2, "k", label=r"Order $h^2$")
plt.legend()
plt.xlabel(r"Mesh width $h$")
plt.ylabel("Relative error")
plt.title("Dirichlet boundary condition")
plt.tight_layout()

In [ ]:
# test of Dirichlet problem and H1-norm error
neumann = False
for omega in omegas:
    print(f"angular freqency = {omega}")
    k = omega / c
    errors = []
    for level in levels:
        m = mesh_hierarchy[level]
        u = exact_solution(m, k, x0)
        uh = solve(m, k, a0, a1, b0, b1, u, neumann)
        rel_err = compute_error(u, uh, norm="h1", quad_deg=4)
        print(f"refinement level {level}, relative error {rel_err:.2%}")
        errors.append(rel_err)
    k = np.polyfit(np.log(hs), np.log(errors), 1)[0]
    print(f"convergence rate = {k:.2}")
    plt.loglog(hs, errors, "-o",
               label=r"Relative error of $\omega=$"+f"{omega}")
    print("----------------------------------------")

plt.loglog(hs, hs, "k", label=r"Order $h$")
plt.legend()
plt.xlabel(r"Mesh width $h$")
plt.ylabel("Relative error")
plt.title("Dirichlet boundary condition")
plt.tight_layout()

In [ ]:
# test of Neumann problem and L2-norm error
# see [1, Figure 13]
neumann = True
for omega in omegas:
    print(f"angular freqency = {omega}")
    k = omega / c
    errors = []
    for level in levels:
        m = mesh_hierarchy[level]
        g = neumann_bc(m, k, x0)
        uh = solve(m, k, a0, a1, b0, b1, g, neumann, quad_deg=5)
        u = exact_solution(m, k, x0)
        rel_err = compute_error(u, uh, quad_deg=4)
        print(f"refinement level {level}, relative error {rel_err:.2%}")
        errors.append(rel_err)
    k = np.polyfit(np.log(hs), np.log(errors), 1)[0]
    print(f"convergence rate = {k:.2}")
    plt.loglog(hs, errors, "-o",
               label=r"Relative error of $\omega=$"+f"{omega}")
    print("----------------------------------------")

plt.loglog(hs, hs**2, "k", label=r"Order $h^2$")
plt.legend()
plt.xlabel(r"Mesh width $h$")
plt.ylabel("Relative error")
plt.title("Neumann boundary condition")
plt.tight_layout()

In [ ]:
# test of Neumann problem and H1-norm error
neumann = True
for omega in omegas:
    print(f"angular freqency = {omega}")
    k = omega / c
    errors = []
    for level in levels:
        m = mesh_hierarchy[level]
        g = neumann_bc(m, k, x0)
        uh = solve(m, k, a0, a1, b0, b1, g, neumann, quad_deg=5)
        u = exact_solution(m, k, x0)
        rel_err = compute_error(u, uh, norm="h1", quad_deg=4)
        print(f"refinement level {level}, relative error {rel_err:.2%}")
        errors.append(rel_err)
    k = np.polyfit(np.log(hs), np.log(errors), 1)[0]
    print(f"convergence rate = {k:.2}")
    plt.loglog(hs, errors, "-o",
               label=r"Relative error of $\omega=$"+f"{omega}")
    print("----------------------------------------")

plt.loglog(hs, hs, "k", label=r"Order $h$")
plt.legend()
plt.xlabel(r"Mesh width $h$")
plt.ylabel("Relative error")
plt.title("Neumann boundary condition")
plt.tight_layout()

## Reference

[1] Bermúdez, A., Hervella-Nieto, L., Prieto, A., Rodríguez, R., 2007. An optimal perfectly matched layer with unbounded absorbing function for time-harmonic acoustic scattering problems. Journal of Computational Physics 223, 469–488. https://doi.org/10.1016/j.jcp.2006.09.018